# Quantum codes do not fix qubit independent errors
#### Authors: J. Lacalle, L. M. Pozo-Coronado, A. L. Fonseca de Oliveira, R. Martin-Cuevas

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## 1. Finding the basis associated with the 5-qubit quantum code

First, we will import the required libraries and set up a few basic variables.

In [2]:
from sympy import symbols, Symbol, Matrix, I, pprint, diff, re, im
from sympy.core.expr import Expr
from sympy.physics.quantum import Dagger
from sympy.physics.quantum.tensorproduct import TensorProduct

num_qubits = 5

Then, we define the generators used by the 5-qubit quantum code $C$. As we will be using the string themselves to perform calculations on the quantum states, the Dirac notation is avoided when defining these variables.

In [3]:
L0 = Symbol('00000') - Symbol('00011') + Symbol('00101') - Symbol('00110') + \
     Symbol('01001') + Symbol('01010') - Symbol('01100') - Symbol('01111') - \
     Symbol('10001') + Symbol('10010') + Symbol('10100') - Symbol('10111') - \
     Symbol('11000') - Symbol('11011') - Symbol('11101') - Symbol('11110')

L1 = - Symbol('00001') - Symbol('00010') - Symbol('00100') - Symbol('00111') - \
     Symbol('01000') + Symbol('01011') + Symbol('01101') - Symbol('01110') - \
     Symbol('10000') - Symbol('10011') + Symbol('10101') + Symbol('10110') - \
     Symbol('11001') + Symbol('11010') - Symbol('11100') + Symbol('11111')

We will now define the Pauli matrices that will be used later on to construct the different discrete errors.

In [4]:
X = Matrix([[0, 1], [1, 0]])
Y = Matrix([[0, -I], [I, 0]])
Z = Matrix([[1, 0], [0, -1]])

The computational basis ($B_c$) can be defined as follows. We will also define a method to build any binary string, from the non-negative integer in decimal basis that it should represent.

In [5]:
def convert_to_binary_string(i: int, num_qubits: int) -> Symbol:
    """
    This method returns the given non-negative integer in decimal basis, expressed as
    a binary string with as many digits as required by the second parameter.

    :param i: Non-negative integer in decimal basis.
    :param num_bits: Length of the resulting binary string.
    :return: Sympy symbol containing the desired string as name.
    """
    binary_string = str(bin(i)[2:])
    binary_string = ''.join(['0' for _ in range(num_qubits - len(binary_string))]) + binary_string
    return Symbol(binary_string)


B_c = []  # Computational basis: [ 00000, 00001, ..., 11111 ].
for i in range(2 ** num_qubits):
    B_c.append(convert_to_binary_string(i, num_qubits))
print('B_c:', B_c)

B_c: [00000, 00001, 00010, 00011, 00100, 00101, 00110, 00111, 01000, 01001, 01010, 01011, 01100, 01101, 01110, 01111, 10000, 10001, 10010, 10011, 10100, 10101, 10110, 10111, 11000, 11001, 11010, 11011, 11100, 11101, 11110, 11111]


The basis associated with the quantum code $C$ ($B$) can be defined as follows, using the Pauli matrices, and aplying them on all the different qubits of each of the quantum states defined as generators, to obtain:
$B = [|0_L\rangle, |1_L\rangle, X_0|0_L\rangle, X_0|1_L\rangle, X_1|0_L\rangle, X_1|1_L\rangle, ..., Z_4|0_L\rangle, Z_4|1_L\rangle]$

We need to define an auxiliary method to apply 1-qubit operation to quantum states, using its string representation to perform the calculation. We will also need another method to obtain which new states could be obtained if we alter one bit of an incoming binary string.

In [6]:
def apply_1qubit_operation(operation: Matrix, target_qubit: int, initial_state: Expr, num_qubits: int) -> Expr:
    """
    This method applies the specified 1-qubit operation to the initial state, on the desired
    target qubit, and returns the resulting quantum state.
    
    :param operation: 2x2 matrix to be applied. 
    :param target_qubit: Qubit on which it should be applied, one of {0, 1, ..., num_qubits - 1}.
    :param initial_state: Initial quantum state, as an expression in terms of elements of the
        computational basis, expressed as binary strings.
    :param num_qubits: Number of qubits of the quantum state.
    :return: New quantum state after applying the operation.
    """
    
    final_state = 0

    for i in range(2 ** num_qubits):
        index = convert_to_binary_string(i, num_qubits)

        alpha = initial_state.coeff(index)
        state_0, state_1 = get_basis_states_one_bit_apart(index, target_qubit)

        initial_qubit_state = Matrix([[1], [0]]) if index.name[target_qubit] == '0' else Matrix([[0], [1]])
        final_qubit_state = operation * initial_qubit_state

        final_state += alpha * (final_qubit_state[0] * state_0 + final_qubit_state[1] * state_1)

    return final_state

def get_basis_states_one_bit_apart(reference_state: Symbol, target: int) -> tuple:
    """
    Method used to get the two binary strings that would be obtained if we modify any
    one bit of an incoming binary string. E.g.: if we receive the string 00000 and the
    integer '1' as target, the two possible outputs will be 00000 and 01000.
    
    :param reference_state: Incoming binary string. 
    :param target: Bit that would be altered, one of {0, 1, ..., length - 1}.
    :return: Tuple with the new two binary strings.
    """
    reference_state = reference_state.name

    return Symbol(reference_state[:target] + '0' + reference_state[target + 1:]), \
           Symbol(reference_state[:target] + '1' + reference_state[target + 1:])

B = [L0, L1]  # Basis associated with the code C.
for operator in [X, Y, Z]:
    for u in range(num_qubits):
        for generator in [L0, L1]:
            B.append(
                apply_1qubit_operation(operator, u, generator, num_qubits)
            )
print('B:', B)

B: [00000 - 00011 + 00101 - 00110 + 01001 + 01010 - 01100 - 01111 - 10001 + 10010 + 10100 - 10111 - 11000 - 11011 - 11101 - 11110, -00001 - 00010 - 00100 - 00111 - 01000 + 01011 + 01101 - 01110 - 10000 - 10011 + 10101 + 10110 - 11001 + 11010 - 11100 + 11111, -00001 + 00010 + 00100 - 00111 - 01000 - 01011 - 01101 - 01110 + 10000 - 10011 + 10101 - 10110 + 11001 + 11010 - 11100 - 11111, -00000 - 00011 + 00101 + 00110 - 01001 + 01010 - 01100 + 01111 - 10001 - 10010 - 10100 - 10111 - 11000 + 11011 + 11101 - 11110, 00001 + 00010 - 00100 - 00111 + 01000 - 01011 + 01101 - 01110 - 10000 - 10011 - 10101 - 10110 - 11001 + 11010 + 11100 - 11111, -00000 + 00011 + 00101 - 00110 - 01001 - 01010 - 01100 - 01111 - 10001 + 10010 - 10100 + 10111 - 11000 - 11011 + 11101 + 11110, 00001 - 00010 + 00100 - 00111 - 01000 - 01011 + 01101 + 01110 + 10000 - 10011 - 10101 + 10110 - 11001 - 11010 - 11100 - 11111, -00000 - 00011 - 00101 - 00110 + 01001 - 01010 - 01100 + 01111 + 10001 + 10010 - 10100 - 10111 - 11000 